In [2]:
#hide
!pip install plotly
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
     

#hide
from fastbook import *
from fastai import *
from fastai.text.all import *
from IPython.display import display,HTML
import plotly.express as px
from fastai.text.all import TextBlock
from fastai.text.all import language_model_learner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 46.1 MB/s eta 0:00:0000:0100:01


In [3]:
# Grab the data from the CSVs.
real = pd.read_csv('news/True.csv')
fake = pd.read_csv('news/Fake.csv')
# Add the column that we are trying to determine if its fake or real
real['real/fake'] = 'Real'
fake['real/fake'] = 'Fake'
#Combine the files
df = pd.concat([real,fake],ignore_index=True)
df.head()

,title,text,subject,date,real/fake
0,"As U.S. budget fight looms, Republicans flip their fiscal script","WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a figh...",politicsNews,"December 31, 2017",Real
1,U.S. military to accept transgender recruits on Monday: Pentagon,"WASHINGTON (Reuters) - Transgender people will be allowed for the first time to enlist in the U.S. military starting on Monday as ordered by federal courts, the Pentagon said on Friday, after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban. Two federal appeals courts, one in Washington and one in Virginia, last week rejected the administration’s request to put on hold orders by lower court judges requiring the military to begin accepting transgender recruits on Jan. 1. A Justice Department official said the administration will not chal...",politicsNews,"December 29, 2017",Real
2,Senior U.S. Republican senator: 'Let Mr. Mueller do his job',"WASHINGTON (Reuters) - The special counsel investigation of links between Russia and President Trump’s 2016 election campaign should continue without interference in 2018, despite calls from some Trump administration allies and Republican lawmakers to shut it down, a prominent Republican senator said on Sunday. Lindsey Graham, who serves on the Senate armed forces and judiciary committees, said Department of Justice Special Counsel Robert Mueller needs to carry on with his Russia investigation without political interference. “This investigation will go forward. It will be an investigation ...",politicsNews,"December 31, 2017",Real
3,FBI Russia probe helped by Australian diplomat tip-off: NYT,"WASHINGTON (Reuters) - Trump campaign adviser George Papadopoulos told an Australian diplomat in May 2016 that Russia had political dirt on Democratic presidential candidate Hillary Clinton, the New York Times reported on Saturday. The conversation between Papadopoulos and the diplomat, Alexander Downer, in London was a driving factor behind the FBI’s decision to open a counter-intelligence investigation of Moscow’s contacts with the Trump campaign, the Times reported. Two months after the meeting, Australian officials passed the information that came from Papadopoulos to their American co...",politicsNews,"December 30, 2017",Real
4,Trump wants Postal Service to charge 'much more' for Amazon shipments,"SEATTLE/WASHINGTON (Reuters) - President Donald Trump called on the U.S. Postal Service on Friday to charge “much more” to ship packages for Amazon (AMZN.O), picking another fight with an online retail giant he has criticized in the past. “Why is the United States Post Office, which is losing many billions of dollars a year, while charging Amazon and others so little to deliver their packages, making Amazon richer and the Post Office dumber and poorer? Should be charging MUCH MORE!” Trump wrote on Twitter. The president’s tweet drew fresh attention to the fragile finances of the Posta...",politicsNews,"December 29, 2017",Real


In [4]:

# Drop unnecessary columns that will not help the model
df = df.drop(['subject', 'date'], axis=1)
df.head()

,title,text,real/fake
0,"As U.S. budget fight looms, Republicans flip their fiscal script","WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a figh...",Real
1,U.S. military to accept transgender recruits on Monday: Pentagon,"WASHINGTON (Reuters) - Transgender people will be allowed for the first time to enlist in the U.S. military starting on Monday as ordered by federal courts, the Pentagon said on Friday, after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban. Two federal appeals courts, one in Washington and one in Virginia, last week rejected the administration’s request to put on hold orders by lower court judges requiring the military to begin accepting transgender recruits on Jan. 1. A Justice Department official said the administration will not chal...",Real
2,Senior U.S. Republican senator: 'Let Mr. Mueller do his job',"WASHINGTON (Reuters) - The special counsel investigation of links between Russia and President Trump’s 2016 election campaign should continue without interference in 2018, despite calls from some Trump administration allies and Republican lawmakers to shut it down, a prominent Republican senator said on Sunday. Lindsey Graham, who serves on the Senate armed forces and judiciary committees, said Department of Justice Special Counsel Robert Mueller needs to carry on with his Russia investigation without political interference. “This investigation will go forward. It will be an investigation ...",Real
3,FBI Russia probe helped by Australian diplomat tip-off: NYT,"WASHINGTON (Reuters) - Trump campaign adviser George Papadopoulos told an Australian diplomat in May 2016 that Russia had political dirt on Democratic presidential candidate Hillary Clinton, the New York Times reported on Saturday. The conversation between Papadopoulos and the diplomat, Alexander Downer, in London was a driving factor behind the FBI’s decision to open a counter-intelligence investigation of Moscow’s contacts with the Trump campaign, the Times reported. Two months after the meeting, Australian officials passed the information that came from Papadopoulos to their American co...",Real
4,Trump wants Postal Service to charge 'much more' for Amazon shipments,"SEATTLE/WASHINGTON (Reuters) - President Donald Trump called on the U.S. Postal Service on Friday to charge “much more” to ship packages for Amazon (AMZN.O), picking another fight with an online retail giant he has criticized in the past. “Why is the United States Post Office, which is losing many billions of dollars a year, while charging Amazon and others so little to deliver their packages, making Amazon richer and the Post Office dumber and poorer? Should be charging MUCH MORE!” Trump wrote on Twitter. The president’s tweet drew fresh attention to the fragile finances of the Posta...",Real


In [5]:
# Standard practice to remove all puncation NLP models because most NLP tasks focus on understanding the meaning of the words in a sentence or document, rather than the punctuation or other formatting. Removing punctuation can help simplify the input text and make it easier for the model to focus on the words themselves.

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

df = df.applymap(lambda x: remove_punctuation(x) if isinstance(x, str) else x)
df.head()

,title,text,real/fake
0,As US budget fight looms Republicans flip their fiscal script,WASHINGTON Reuters The head of a conservative Republican faction in the US Congress who voted this month for a huge expansion of the national debt to pay for tax cuts called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018 In keeping with a sharp pivot under way among Republicans US Representative Mark Meadows speaking on CBS’ “Face the Nation” drew a hard line on federal spending which lawmakers are bracing to do battle over in January When they return from the holidays on Wednesday lawmakers will begin trying to pass a federal budget in a fight likely to be l...,Real
1,US military to accept transgender recruits on Monday Pentagon,WASHINGTON Reuters Transgender people will be allowed for the first time to enlist in the US military starting on Monday as ordered by federal courts the Pentagon said on Friday after President Donald Trump’s administration decided not to appeal rulings that blocked his transgender ban Two federal appeals courts one in Washington and one in Virginia last week rejected the administration’s request to put on hold orders by lower court judges requiring the military to begin accepting transgender recruits on Jan 1 A Justice Department official said the administration will not challenge those ...,Real
2,Senior US Republican senator Let Mr Mueller do his job,WASHINGTON Reuters The special counsel investigation of links between Russia and President Trump’s 2016 election campaign should continue without interference in 2018 despite calls from some Trump administration allies and Republican lawmakers to shut it down a prominent Republican senator said on Sunday Lindsey Graham who serves on the Senate armed forces and judiciary committees said Department of Justice Special Counsel Robert Mueller needs to carry on with his Russia investigation without political interference “This investigation will go forward It will be an investigation conducted ...,Real
3,FBI Russia probe helped by Australian diplomat tipoff NYT,WASHINGTON Reuters Trump campaign adviser George Papadopoulos told an Australian diplomat in May 2016 that Russia had political dirt on Democratic presidential candidate Hillary Clinton the New York Times reported on Saturday The conversation between Papadopoulos and the diplomat Alexander Downer in London was a driving factor behind the FBI’s decision to open a counterintelligence investigation of Moscow’s contacts with the Trump campaign the Times reported Two months after the meeting Australian officials passed the information that came from Papadopoulos to their American counterparts ...,Real
4,Trump wants Postal Service to charge much more for Amazon shipments,SEATTLEWASHINGTON Reuters President Donald Trump called on the US Postal Service on Friday to charge “much more” to ship packages for Amazon AMZNO picking another fight with an online retail giant he has criticized in the past “Why is the United States Post Office which is losing many billions of dollars a year while charging Amazon and others so little to deliver their packages making Amazon richer and the Post Office dumber and poorer Should be charging MUCH MORE” Trump wrote on Twitter The president’s tweet drew fresh attention to the fragile finances of the Postal Service at a ti...,Real


In [42]:
print(df['real/fake'].value_counts())

Fake    23481
Real    21417
Name: real/fake, dtype: int64


In [43]:
# We will be using a pretrain'ed fastAI model that has been trained on wikipedia.
# We will train this model to to attempt to predict the next word of news article
# Then we will train it to categorize between fake and real

In [6]:
dls_lm = DataBlock(
    blocks=TextBlock.from_df('text', is_lm=True),
    get_x=ColReader('text'),
    splitter=RandomSplitter(0.1)
).dataloaders(df, bs=128, seq_len=80)

In [50]:
dls_lm.show_batch(max_n=3)


,text,text_
0,xxbos xxup washington xxmaj reuters xxmaj former xxmaj new xxmaj york xxmaj mayor xxmaj michael xxmaj bloomberg said on xxmaj monday he would not mount an independent bid for the xxup us presidency because he feared it would increase the chances that xxmaj republicans xxmaj donald xxmaj trump or xxmaj ted xxmaj cruz could end up in the xxmaj white xxmaj house a billionaire media mogul who combined businessfriendly fiscal policies with liberal views on gun control and other social,xxup washington xxmaj reuters xxmaj former xxmaj new xxmaj york xxmaj mayor xxmaj michael xxmaj bloomberg said on xxmaj monday he would not mount an independent bid for the xxup us presidency because he feared it would increase the chances that xxmaj republicans xxmaj donald xxmaj trump or xxmaj ted xxmaj cruz could end up in the xxmaj white xxmaj house a billionaire media mogul who combined businessfriendly fiscal policies with liberal views on gun control and other social issues
1,the first islands xxmaj richards said his bookings have already dropped for the xxmaj caribbean his company s secondmost popular destination worldwide for xxup us tourists xxmaj more than 26 million international tourists were expected this year in the xxmaj caribbean the xxmaj world xxmaj travel and xxmaj tourism xxmaj council xxup xxunk projected before xxmaj irma hit xxmaj in 2016 foreign visitors spent 31 billion in the xxmaj caribbean and were expected to spend an additional 53 percent in,first islands xxmaj richards said his bookings have already dropped for the xxmaj caribbean his company s secondmost popular destination worldwide for xxup us tourists xxmaj more than 26 million international tourists were expected this year in the xxmaj caribbean the xxmaj world xxmaj travel and xxmaj tourism xxmaj council xxup xxunk projected before xxmaj irma hit xxmaj in 2016 foreign visitors spent 31 billion in the xxmaj caribbean and were expected to spend an additional 53 percent in 2017
2,officials to discuss setting up a mentorship program there xxmaj the program never happened but school officials alerted parents to the fact that he had been inside the school during regular school hours when their young children also would have been in xxunk schools are required to screen all volunteers and employees during the interview process the law doesn t require every person who enters onto school property to have a background check xxunk background checks for church workers and,to discuss setting up a mentorship program there xxmaj the program never happened but school officials alerted parents to the fact that he had been inside the school during regular school hours when their young children also would have been in xxunk schools are required to screen all volunteers and employees during the interview process the law doesn t require every person who enters onto school property to have a background check xxunk background checks for church workers and volunteers


In [56]:
from fastai.text.all import AWD_LSTM
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()
learn.fit_one_cycle(1, 2e-2)


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.875571,3.792140,0.319298,44.351231,33:44


In [57]:
learn.save_encoder('finetuned')


In [61]:
# Lets see our next word predictor create some sentences!
TEXT = "Breaking News"
N_WORDS = 100
N_SENTENCES = 1
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

In [15]:
# Now that our pretrained model is done, lets create a classifier for our fake/real news.
dls_clas = DataBlock(
    blocks=(TextBlock.from_df('text', vocab=dls_lm.vocab), CategoryBlock),
    #get_x=Pipeline([lambda x: (x['title'], x['text'])]),
    get_x = ColReader('text'),
    get_y=ColReader('real/fake'),
    splitter=RandomSplitter(0.2)
).dataloaders(df, bs=128, seq_len=72)

In [16]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [17]:
learn = learn.load_encoder('finetuned')

In [18]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.069189,0.005632,0.998441,04:10


In [19]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.029934,0.002406,0.999220,06:33


In [20]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.020836,0.003007,0.999443,08:24
1,0.025026,0.002496,0.999332,08:25


In [22]:
text = "For centuries politicians in this country assumed that in order to win an election you had to convince voters to vote for you You had to make their case That was called democracyBut the defining strategic insight of the modern Democratic Party is they dont really need to convince anyone of anything What matters is demographics To import enough people from elsewhere people who are financially dependent on you in order to live Thats the story of the state of California Thats why California changed Its why Texas is changing but its also happening at the local level all over the country Democrats have realized that they win the vote in virtually every major city but the suburbs remain contested and that matters because suburban voters like school boards and state legislators and of course members of Congress  So if you want total control over the entire country you need demographic change everywhere including in the suburbs This is a very obvious insight and Democrats have been on the case for quite a while Back in 2009 the Obama administration took the county of Westchester in New York a solidly Democratic place to court Why did they do that The administration withheld tens of millions of dollars in federal funding unless Westchester pledged to build 10000 lowincome highdensity apartments in their suburbs "
pred = learn.predict(text)
print(pred[0]) # predicted label (Real or Fake)
print(pred[2][pred[1].item()]) # predicted probability for the predicted label

Fake
tensor(0.9999)


In [23]:
#NYTIMES
text = "HOUSTON  Representative Colin Allred a Dallasarea Democrat who defeated an incumbent Republican to gain his seat in 2018 announced on Wednesday that he would challenge Senator Ted Cruz of Texas next year In a threeminute video Mr Allred 40 a former civil rights lawyer who played as a linebacker in the NFL presented himself as a bipartisan politician whose workingclass upbringing would enable him to overcome the long odds No Democrat has won statewide office in Texas since the 1990s We dont have to be embarrassed by our senator he said after describing Mr Cruz as someone who cheered on the mob during the Capitol riot and who left Texas to go to the resort city of Cancun Mexico during the 2021 winter storm and power grid failure that killed hundreds of Texans We can get a new one Mr Allred came into office riding a wave of Democratic enthusiasm that nearly unseated Mr Cruz during his last reelection fight a 2018 victory over Beto ORourke then a littleknown representative from El Paso Mr ORourke lost by about 25 percentage points a thin margin in the Republicandominated state The same year Mr Allred defeated Representative Pete Sessions a Republican in a Dallasarea district that has since been redrawn to become more favorable for Democrats Almost from the start Mr Allred has shown an ability to attract interest from donors outraising Mr Sessions and continuing to demonstrate the kind of strong fundraising ability that would be necessary in a statewide race in Texas Mr Cruz is highly unpopular among Texas Democrats but he has so far survived all attempts to oust him Enthusiasm is also low among many Texas Democrats who watched Mr ORourke lose badly to Gov Greg Abbott last year despite his wellfunded campaign And Mr Allred whose decision to enter the race began emerging in news reports before Wednesdays announcement has seen expectations for his campaign set low The magazine Texas Monthly suggested that he was a replacementlevel candidate In other words as good as any other Democrat but not a star Editors Picks Restaurant Review Naro Makes New Magic From Classic Korean Dishes My Dead Father Had a Secret Do I Tell Our Family Gordon Lightfoots 10 Essential Songs Nick Maddux a spokesman for Mr Cruzs campaign described Mr Allred as a farleft radical in a statement on Wednesday His voting record is completely outoftouch with Texas he said For over a decade Sen Cruz has been leading the fight for jobs freedom and security in Texas Mr Allreds announcement video acknowledged that he was a long shot presenting himself as an underdog who never knew his father and pulled himself up into elite football law school and Congress He said he would focus on Texas issues not divisive cultural ones discussing rural hospital closures and prescription drug prices in his video As for Mr Cruz he said All hat no cattle"
pred = learn.predict(text)
print(pred[0]) # predicted label (Real or Fake)
print(pred[2][pred[1].item()]) # predicted probability for the predicted label

Fake
tensor(0.9717)


In [24]:
#NYTIMES
text = "Federal Reserve officials raised interest rates by a quarter-point on Wednesday in the tenth straight move in their fight against rapid inflation — but they also opened the door to a possible pause in rate increases. Central bankers lifted rates to a range of 5 percent to 5.25 percent, a level they have not reached since the summer of 2007. The move capped the fastest series of rate increases since the 1980s, as the central bank led by Chair Jerome H. Powell attempts to slow the economy and weigh down price increases. Here’s what to know about the central bank’s latest move: In their statement announcing the decision, Fed policymakers indicated that they will watch to see whether future rate moves are necessary. That marks a shift in stance: For months, they had assumed that additional increases would be needed. This opens the door to a possible pause in rate increases. “In determining the extent to which additional policy firming may be appropriate to return inflation to 2 percent over time, the committee will take into account the cumulative tightening of monetary policy, the lags with which monetary policy affects economic activity and inflation, and economic and financial developments,” the policy-setting Federal Open Market Committee said in its release. The “may” in the statement is a signal that officials are not confident that further moves will absolutely be necessary. In his news conference after the rate announcement, Mr. Powell said that “a decision on a pause was not made today.” He added that “we’ll approach that question at the June meeting,” punting the decision on a potential pause for another month. Investor bets on where interest rates go from here are tilted toward a pause at the Fed’s next meeting and then lower interest rates later in the year. (Mr. Powell said that cutting interest rates this year was “not in our forecast.”) One calculation put the likelihood that the Fed would hold off changing rates when it meets in June at 80 percent. The Fed does not “have an agenda to further consolidate banks,” Mr. Powell said, in response to a question about the seizure and sale of First Republic Bank to JPMorgan Chase, already the country’s largest bank. Mr. Powell said it was a good policy to limit the expansion of the biggest banks, but that exceptions made sense in cases of bank failures. He also said that credit conditions were tightening following the banking turmoil, which could slow the economy. Mr. Powell said it was “essential” that the nation’s debt limit is raised in a timely way. “It’s very important that this be done,” he added, noting that the Fed does not give advice to politicians. The consequences for the U.S. economy of a debt default could be dire, he said. “We shouldn’t even be talking about a world in which the U.S. doesn’t pay its bills,” he said. "
pred = learn.predict(text)
print(pred[0]) # predicted label (Real or Fake)
print(pred[2][pred[1].item()]) # predicted probability for the predicted label

Real
tensor(0.9985)


In [25]:
#NYTIMES
text = "The United Auto Workers a politically potent labor union is planning to withhold its endorsement of President Biden in the early stages of the 2024 race according to an internal memo from its president to members on Tuesday The memo written by Shawn Fain the Detroitbased unions president said the leadership of the United Auto Workers had traveled to Washington last week to meet with Biden administration officials and had expressed our concerns with the electric vehicle transition that the president has pursued The memo underscores how some of Mr Bidens boldest moves to fight climate change which animate his liberal base could at the same time weaken his political support among another crucial constituency The UAW has shrunk in size in recent decades but it still counts about 400000 active members with a robust presence in Michigan a critical battleground state for Democrats In April the Biden administration proposed the nations most ambitious climate regulations yet which would ensure that twothirds of new passenger cars are allelectric by 2032  up from just 58 percent today The rules if enacted could sharply lower planetwarming pollution from vehicle tailpipes the nations largest source of greenhouse emissions But they come with costs for autoworkers because it takes fewer than half the laborers to assemble an allelectric vehicle as it does to build a gasolinepowered car In the memo Mr Fain provided talking points for members about why the union was not immediately lining up behind Mr Biden writing that if companies received federal subsidies then workers must be compensated with top wages and benefits Whos Running for President in 2024 Card 1 of 10 The race begins Four years after a historically large number of candidates ran for president the field for the 2024 campaign is starting out small and is likely to be headlined by the same two men who ran last time President Biden and Donald Trump Heres who has entered the race so far and who else might run President Biden The president has cast himself as a protector of democracy and a stabilizing force after the upheaval of the Trump administration Biden is running for reelection as the oldest person ever to hold the presidency a subject of concern among many Democrats though the party has publicly set aside those worries and rallied around him Donald Trump The former president is running to retake the office he lost in 2020 Though somewhat diminished in influence within the Republican Party  and facing several legal investigations  he retains a large and committed base of supporters and he could be aided in the primary by multiple challengers splitting a limited antiTrump vote Nikki Haley The former governor of South Carolina and UN ambassador under Trump has presented herself as a member of a new generation of leadership and emphasized her life experience as a daughter of Indian immigrants She was long seen as a rising GOP star but her allure in the party has declined amid her onagain offagain embrace of Trump Asa Hutchinson The former governor of Arkansas is one of a relatively small number of Republicans who have been openly critical of Trump Hutchinson has denounced the former presidents efforts to overturn the 2020 election and said Trump should drop out of the presidential race Marianne Williamson The selfhelp author and former spiritual adviser to Oprah Winfrey is running for a second time In her 2020 campaign the Democrat called for a federal Department of Peace supported reparations for slavery and called Trumpism a symptom of an illness in the American psyche that could not be cured with political policies Vivek Ramaswamy The multimillionaire entrepreneur and author describes himself as antiwoke and is known in rightwing circles for opposing corporate efforts to advance political social and environmental causes He has never held elected office and does not have the name recognition of most other GOP contenders Larry Elder The conservative talk radio host who was a breakout star for the right after running unsuccessfully in Californias recall election in 2021 announced on Tucker Carlsons show on Fox News that he was running for president saying that he had a moral religious and a patriotic duty to give back to a country thats been so good to my family and me Robert Kennedy The Democrat a longtime vaccine skeptic and a member of the Kennedy political dynasty declared that he would challenge President Biden for the Democratic nomination in a longshot bid for the White House Others who might run Gov Ron DeSantis of Florida former Vice President Mike Pence Senator Tim Scott of South Carolina and Gov Chris Sununu of New Hampshire are seen as weighing Republican bids for the White House The EV transition is at serious risk of becoming a race to the bottom the memo reads referring to electric vehicles We want to see national leadership have our back on this before we make any commitments Mr Fain won the UAW presidency as an insurgent candidate this year toppling the incumbent Ray Curry Mr Fain promised a more confrontational path ahead of contract talks In the memo he notes that 150000 autoworkers are fighting for a new contract with the socalled Big Three auto companies in September writing Well stand with whoever stands with our members in that fight Labor support is a key part of Mr Bidens political coalition and his portrayal of himself as a fighter for the middle class Within hours of Mr Bidens formal entry into the 2024 race a number of top labor unions backed Mr Biden including the Amalgamated Transit Union the Service Employees International Union and the International Brotherhood of Electrical Workers Editors Picks Restaurant Review Naro Makes New Magic From Classic Korean Dishes Its the End of a World as We Know It Gordon Lightfoots 10 Essential Songs Several national unions were quick to endorse Mr Fain wrote in his memo The United Auto Workers is not yet making an endorsement Mr Bidens campaign trumpeted his support from other labor unions in a news release Notably Mr Bidens first public appearance after announcing his reelection campaign last week was addressing a labor conference in the nations capital Ive said many times Wall Street didnt build America he told the cheering union crowd last week The middle class built America and unions built the middle class The United Auto Workers which has historically endorsed Democrats and supported Mr Biden in 2020 makes clear in the memo that it has no intent of backing the Republican frontrunner former President Donald J Trump Withholding a formal endorsement for now instead appears to be a bid for leverage or concessions from the administration Another Donald Trump presidency would be a disaster reads Mr Fains memo which was first reported by The Detroit News But our members need to see an alternative that delivers real results We need to get our members organized behind a proworker proclimate and prodemocracy political program that can deliver for the working class Mr Biden has sought to accelerate the transition to allelectric vehicles as a centerpiece of his effort to tackle climate change A 2021 report by the International Energy Agency found that nations would have to stop sales of new gasolinepowered cars by 2035 to avert the deadliest effects of a warming planet To help reach that goal Mr Biden has pushed a fleet of policies designed to promote electric vehicles The Biden administrations proposed climate regulations announced in April are designed to add legal teeth to consumer incentives compelling automakers to manufacture and sell more electric vehicles The Environmental Protection Agency rules however are not yet final They are open for public comment and could still be weakened or otherwise changed before being completed next year As the Biden administration prepared to unveil the new clean car rules last month officials planned for Michael S Regan the head of the EPA to announce the policies in Detroit surrounded by Americanmade allelectric vehicles But as auto executives and the United Auto Workers learned the details of the proposed regulations some grew uneasy about publicly supporting it according to two people familiar with their thinking No one from the United Auto Workers attended the unveiling according to the organizations spokesman although representatives from Ford General Motors and MercedesBenz were there And the setting was moved from Detroit to the EPA headquarters in Washington"
pred = learn.predict(text)
print(pred[0]) # predicted label (Real or Fake)
print(pred[2][pred[1].item()]) # predicted probability for the predicted label

Fake
tensor(0.9997)


In [27]:
#FOX NEWS
text = "President Joe Biden and the White House continue to ignore questions about the Delaware Democrats grandchild as son Hunter Biden appeared in an Arkansas court this week to face a paternity case involving his 4yearold daughter On Tuesday White House press secretary Karine JeanPierre rebuffed a reporters question about Bidens seventh grandchild Navy Joan Roberts A reporter from the Daily Mail asked about Hunters paternity case in Independence County Arkansas questioning whether the president and first lady Jill Biden are monitoring the case and followed up by asking why neither Biden has yet acknowledged Navy Im not going to speak to that from here JeanPierre said before quickly moving on as a Why not could be heard from the press corps 2011 EMAILS REVEAL HUNTER BIDEN HELPED BUSINESS ASSOCIATES GET ACCESS TO VP BIDEN TOP AIDE A sketch shows Independence County Circuit Judge Holly Meyer presiding over the child support hearing with Hunter Biden left and his lawyer Abbe Lowell on May 1 2023 John Kushmaul Navys mother Lunden Alexis Roberts 31 met the now52yearold while working as a stripper at a District of Columbia gentlemens club while Hunter was dating Hallie Biden  the widow of his late brother Joseph R Beau Biden III  according to the New York Post Roberts also reportedly wants Navy to be able to use the Biden surname Hunter Biden continues to litigiously fight the name change as well On Fox News America Reports Ari Fleischer  who served as press secretary for President George W Bush  was astounded at the White Houses continued refusal to address Bidens seventh grandchild or Hunters paternity suit It is a bigger story And the story really gets to the heart of Joe Biden and Joe Bidens lies about what his son Hunter Biden did in all kinds of things but mostly in his business dealings where Joe Biden said he didnt know anything Fleischer said EXCIA MOSCOW CHIEF REFUSED TO SIGN HUNTER LAPTOP LETTER NO EVIDENCE PUTIN INVOLVED Hunter Biden gets off plane with president Joe left and Hunter Biden AP Photo  Patrick Semansky  File This is a White House that rallies around the presidents son I understand that But theyre rallying around him to protect the president  so the presidents lies dont come out Fleischer also said court cases lead to sworn statements that have a way of undermining White Houses Host Bill Hemmer noted that Biden left Navy out of a recent numeric reference to his grandchildren remarking during Take Your Child to Work Day that he only has six grandchildren Fleischer added he was reminded of when he was bombarded with questions after Jenna Bush was cited in Austin Texas in 2001 for alcohol possession by a minor  and contrasted the press reaction then and now Jenna was a teenager a freshman in college had an underage drink at a bar and it was a setup She got arrested by the police I got more questions about that than Joe Biden has ever gotten questions from this White House press corps about the behavior of his adult son Fleischer said And that just shows you as you indicated earlier President Biden acts as if he doesnt need the press its because he already has the press"
pred = learn.predict(text)
print(pred[0]) # predicted label (Real or Fake)
print(pred[2][pred[1].item()]) # predicted probability for the predicted label

Fake
tensor(1.0000)
